# Run the network several times with weights from LHS

In [1]:
# automatically apply changes in netgen
%reload_ext autoreload
%autoreload 2

In [2]:
import os, sys
import pandas as pd
import numpy as np
import random
import math
from future.utils import listvalues
from copy import deepcopy

import cbgt.netgen as ng
import cbgt.generateFrames as gf
#import cbgt.analyzefx as afx
#from cbgt import vis, sim

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.simplefilter('ignore', np.RankWarning)
warnings.filterwarnings("ignore", module="matplotlib")
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
clrs = ['#347fff', '#febe08']
eclrs = ['#1657de', '#f5a005']

rc = {'font.family': u'sans-serif',
'font.sans-serif': [u'Helvetica',  u'Arial', u'sans-serif'],
'lines.solid_capstyle': u'butt'}
sns.set(style='ticks', font='Helvetica', rc=rc, font_scale=1.4)
# splot.style(theme='safari-day', grid=True, spines=True, fscale=1.4, figsize=(6., 3.5))
# bdf = pd.read_csv("/Users/kyle/demo_cbgt_bdf.csv")

# Specific for the HDDM study
#import hddm

# Save/load datatables functions

In [3]:
# directory = the filepath to where you want to save your data.  
#   You could use directory="" to save it in the same directory 
#   you are running the notebook in.

# foldername = a name for a new folder to put this specific data set in.
#   The code will create this folder for you.  
#   Whenever you have a new data set, you need to use a new foldername, 
#   otherwise it will overwrite the older results.

def saveData(paramtable,datatables,directory,foldername):
    targetfolder = os.path.join(directory,foldername)
    if not os.path.exists(targetfolder):
        os.makedirs(targetfolder)
    file_name = os.path.join(targetfolder,"param.pkl")
    paramtable.to_pickle(file_name,protocol=0)
    for i in range(len(datatables)):
        file_name = os.path.join(targetfolder,"data"+str(i)+".pkl")
        datatables[i].to_pickle(file_name,protocol=0)
        
def loadData(directory,foldername):
    targetfolder = os.path.join(directory,foldername)
    file_name = os.path.join(targetfolder,"param.pkl")
    paramtable = pd.read_pickle(file_name)
    filecount = len([f for f in os.listdir(targetfolder) 
                     if os.path.isfile(os.path.join(targetfolder, f)) 
                     and f.startswith("data") and f.endswith(".pkl")])
    datatables = []
    for i in range(filecount):
        file_name = os.path.join(targetfolder,"data"+str(i)+".pkl")
        datatables.append(pd.read_pickle(file_name))
    return (paramtable,datatables)

# Global Parameters:

In [4]:
# Working directory
parentDir = 'dftest'
savedir = os.path.join(os.path.expanduser('~'), parentDir)
if not os.path.isdir(savedir):
    os.mkdir(savedir)
saveTest=os.path.join(savedir, 'test')
ng.setDirectory(saveTest)

In [5]:
BaseStim = 0
Stim = 2.54
Dynamic = 30.0 # Thalamic baseline
Choices = 2
rampingCTX = True
popscale = .3
Start=600

presetNo = ng.set_post_learning_weights(dMSN=[1.0, 1.0], iMSN=[1.0,1.0])
presetLow = ng.set_post_learning_weights(dMSN=[1.01, 0.99], iMSN=[1.0,1.0])
presetHi = ng.set_post_learning_weights(dMSN=[1.03, 0.98], iMSN=[1.0,1.0])
presets = [presetNo, presetLow, presetHi]

conds = ['no', 'low', 'high']
condABC = ['z', 'a', 'b']
cond12 = [0, 1, 2]

savedirs = [os.path.join(savedir, c) for c in conds]
saveNo, saveLow, saveHi = savedirs

presetDict = dict(zip(conds, presets))
condOrder = dict(zip(conds, condABC))
condLevel = dict(zip(conds+['test'], cond12+[0]))

# Efficacy sampling f(x) for individual networks

In [8]:
def get_sampling_params(scale=.05, values=[0.018,0.018,0.018,0.018,
                                           0.0986,0.0986,0.0986,0.0986,
                                           0.28,1.1,0.28,1.65,0.35,0.018,0.0114,0.117,
                                           0.255,0.255,0.03,0.132,0.07,1.51,0.03,1.75]):
    
    Cx_dMSN_NMDA_L = values[0,0] #0.063
    Cx_dMSN_NMDA_R = values[0,1] #0.063
    Cx_iMSN_NMDA_L = values[0,2] #0.063
    Cx_iMSN_NMDA_R = values[0,3] #0.063
    
    Cx_dMSN_AMPA_L = values[0,4] #0.1286
    Cx_dMSN_AMPA_R = values[0,5] #0.1286
    Cx_iMSN_AMPA_L = values[0,6] #0.1286
    Cx_iMSN_AMPA_R = values[0,7] #0.1286
    
    dMSN_iMSN = values[0,8] #0.28
    D1STR_GPi = values[0,9] #1.10
    
    iMSN_dMSN  = values[0,10] #0.28
    D2STR_GPeP = values[0,11] #1.65
    
    GPe_STN = values[0,12] #0.4
    GPe_GPi = values[0,13] #0.012
    
    STN_GPi    = values[0,14] #0.0324
    
    GPi_Th     =  values[0,15] #0.067
    
    Th_STRd = values[0,16] #0.255
    Th_STRi = values[0,17] #0.255
    
    
    
    CxTh       =    values[0,18] #0.03
    CxFSI      =    values[0,19] #0.132
    STN_GPeP_A =    values[0,20] #0.07
    STN_GPeP_N =    values[0,21] #4.01
    ThCx       =    values[0,22] #0.02
    
    GPeP_GPeP  =    values[0,23] #1.5 
        
    STN_GPeP = [STN_GPeP_A, STN_GPeP_N] # [AMPA, NMDA]
    
    CxSTRd = (Cx_dMSN_NMDA_L+Cx_dMSN_NMDA_R)/2.0
    CxSTRi = (Cx_iMSN_NMDA_L+Cx_iMSN_NMDA_R)/2.0
    
    preset = ng.set_post_learning_weights(dMSN=[Cx_dMSN_NMDA_L/CxSTRd, Cx_dMSN_NMDA_R/CxSTRd],
                                          iMSN=[Cx_iMSN_NMDA_L/CxSTRi, Cx_iMSN_NMDA_R/CxSTRi])
    
    preset[0]['D1STR_0_0'] = {'dpmn_w':Cx_dMSN_AMPA_L}
    preset[0]['D1STR_1_0'] = {'dpmn_w':Cx_dMSN_AMPA_R}
    preset[0]['D2STR_0_0'] = {'dpmn_w':Cx_iMSN_AMPA_L}
    preset[0]['D2STR_1_0'] = {'dpmn_w':Cx_iMSN_AMPA_R}
    
    #print(preset)

    
    mu = dict(Cx={'STRd':CxSTRd, 'STRi':CxSTRi, 'Th':CxTh, 'FSI':CxFSI}, 
                Th={'STRd':Th_STRd, 'STRi':Th_STRi, 'Cx':ThCx},
                D1STR={'GPi': D1STR_GPi, 'D2STR': dMSN_iMSN},
                D2STR={'GPeP': D2STR_GPeP, 'D1STR_syn': iMSN_dMSN},
                STN={'GPi': STN_GPi, 'GPeP': STN_GPeP},
                GPi={'Th': GPi_Th},
                GPeP={'STN': GPe_STN, 'GPi': GPe_GPi, 'GPeP': GPeP_GPeP})

    # PROVISIONAL
    sd = dict(Cx={'STRd':CxSTRd, 'STRi':CxSTRi, 'Th':CxTh, 'FSI':CxFSI}, 
                Th={'STRd':Th_STRd, 'STRi':Th_STRi, 'Cx':ThCx},
                D1STR={'GPi': D1STR_GPi, 'D2STR': dMSN_iMSN},
                D2STR={'GPeP': D2STR_GPeP, 'D1STR_syn': iMSN_dMSN},
                STN={'GPi': STN_GPi, 'GPeP': STN_GPeP},
                GPi={'Th': GPi_Th},
                GPeP={'STN': GPe_STN, 'GPi': GPe_GPi, 'GPeP': GPeP_GPeP})
    
    for x in list(sd):
        for y in sd[x]:
            sd[x][y]=0
    sd['STN']['GPeP']=[0,0]
    
    #sd = {i:{j: mu[i][j]*scale for j in list(mu[i])} for i in list(mu)}
    return mu, sd, preset


def sample_network_efficacies(muEff, sdEff, N):
    X = {}
    nuclei = list(muEff)
    for i in nuclei:
        targets = list(muEff[i])
        X[i] = {}
        for j in targets:
            X[i][j] = np.random.normal(muEff[i][j], sdEff[i][j], N)
    X['STN']['GPeP']=np.array(muEff['STN']['GPeP'])
    return X


def build_single_network(X, idx=0):
    
    Cx={'STRd': X['Cx']['STRd'][idx], 
        'STRi': X['Cx']['STRi'][idx],
        'Th': X['Cx']['Th'][idx], 
        'FSI': X['Cx']['FSI'][idx]}
    
    D1STR={'GPi': X['D1STR']['GPi'][idx],
           'D2STR': X['D1STR']['D2STR'][idx]}
    D2STR={'GPeP': X['D2STR']['GPeP'][idx],
           'D1STR_syn': X['D2STR']['D1STR_syn'][idx]}
    
    STN={'GPi': X['STN']['GPi'][idx],
        'GPeP': [X['STN']['GPeP'][0],X['STN']['GPeP'][1]]}
    
    GPi={'Th': X['GPi']['Th'][idx]}
    
    Th={'STRd': X['Th']['STRd'][idx], 
        'STRi': X['Th']['STRi'][idx],
        'Cx': X['Th']['Cx'][idx]}
    
    GPeP={'STN': X['GPeP']['STN'][idx],
          'GPi': X['GPeP']['GPi'][idx],
          'GPeP': X['GPeP']['GPeP'][idx]}

    return ng.getConEff(Cx=Cx, D1STR=D1STR, D2STR=D2STR, STN=STN, GPi=GPi, Th=Th, GPeP=GPeP)

# Latin Hipercub Sampling - uniformly distributed values

In [7]:
import lhsmdu
import matplotlib.pyplot as plt
import numpy

# Limits per each weight
xlimits = np.array([[0.01, 0.022], [0.01, 0.022], [0.013, 0.059], [0.013, 0.059],
                    [0.03, 0.64], [0.03, 0.64], [0.03, 0.59], [0.03, 0.59],
                    [0.28, 0.28], [0.22, 1.27], [0.28, 0.28], [1.0, 1.81],
                    [0.33, 0.39], [0.0172, 0.02], [0.0108, 0.0115], [0.08, 0.126],
                    [0.001, 0.261], [0.235, 0.265], [0.007, 0.047], [0.127,0.42],
                    [0.05, 0.1], [1.51, 1.51], [0.021, 0.035], [1.75,1.75]])

# Latin Hypercube Sampling 
numVar=24 
N=300 
values = lhsmdu.sample(numVar,N) # generate Latin Hypercube Sampling of 'numVar' variables, and 'N' samples each.

# Set the correspondig limits per each sample
for i in range(0,numVar):
    xmin=xlimits[i,0]
    xmax=xlimits[i,1]
    values[i,:]=(xmax-xmin)*values[i,:]+xmin
values=np.transpose(values)

In [8]:
# Save the weights dataFrame
df = pd.DataFrame(values)
df.to_csv ("weights.csv")

# Phenotypes sims

In [27]:
import statistics

N_subj = 1
sdScale = .0 # variablity on the efficacy values.
conProb = ng.getConProb()

# number of different phenotypes to explore. 
# It must be = or less than variable num in LHS function
NumSteps=300

cores=4
np.random.seed(seed=np.random.randint(0,1000))

# set stimulus strength
stim = 2.5

# number of simulation we would like to run per each sample of weights (each simulation perform 300 different trials)
totalSim = 1 

for ns in range(0,NumSteps):
    valuesUsed=values[ns,:]
    print('Step', ns, 'Values considered', valuesUsed)
    
    # define parameters
    muEff, sdEff, preset = get_sampling_params(sdScale,valuesUsed)
    X = sample_network_efficacies(muEff, sdEff, N=N_subj)
    subj_eff_dicts = [build_single_network(X, idx=i) for i in range(N_subj)]
    
    # get conn. efficacies for 1st subject network (idx=0)
    idx = 0
    conEff_i = subj_eff_dicts[idx]

    # configure trial and network parameters
    sweepcount = ng.configureSweep(0, experiment='mc', preset=preset, Start=Start,
                               popscale=popscale, BaseStim=BaseStim, Choices=Choices, 
                               WrongStim=stim, RightStim=stim, Dynamic=Dynamic,
                               rampingCTX=True,
                               conProb=conProb, 
                               conEff=conEff_i,
                               seed=list(range(0,totalSim)),
                               d1aw=[0], # Changes the alpha_w for D1
                               d2aw=[0], # Changes the alpha_w for D2
                               rewardprob=[1.0], # It's overwritten inside the netgen.
                               dpmn_ratio=0.,   # it doesn't do anything
                               dpmn_implied=0.,  # it doesn't do anything
                               SustainedFraction = 0.75
                              )
    sweepcount
    
    # run the simulation
    ng.compileAndRunSweepALL_Onofre(1,0,sweepcount,cores)
    
    results = ng.readAllTrialResults(1,0,totalSim,['popfreqs'])
    decisions = []
    for sn in range(totalSim):
        decisions.append([])
        for result in results[sn]:
            decisions[sn].append(ng.findOutputs2(result)['decision made'])
                
    # Change the None values in decisions' list for 0's
    for xdec in decisions[0][0]:
        if xdec['delay']==None:
            xdec['delay']=0.0
    
    (paramtable,datatables) = gf.generateFrames(results,decisions)
    foldername="Uniform_Test"+str(ns)
    saveData(paramtable,datatables,"",foldername)
    
    
    #Check Reaction Times (RT) in [60,300]
    dd=datatables[0].decisionduration
    if max(dd) >= 300 or min(dd) <= 60: # to be correct, the action decision time must be greater than 70ms
        print('The step', ns, 'causes a reaction time out of bounds')
        # print('min:', min(dd), '& max:', max(dd),'& mean:', statistics.mean(dd), '\\')

    #Check Firing Rates in [0,MAX]
    if max(result['popfreqs'].D1STR_0_0[100:])>40 or max(result['popfreqs'].D1STR_1_0[100:])>40 or max(result['popfreqs'].D2STR_0_0[100:])>40 or max(result['popfreqs'].D2STR_1_0[100:])>40 or max(result['popfreqs'].GPeP_0_0[100:])>150 or max(result['popfreqs'].GPeP_1_0[100:])>150 or max(result['popfreqs'].GPi_0_0[100:])>150 or max(result['popfreqs'].GPi_1_0[100:])>150 or max(result['popfreqs'].STNE_0_0[100:])>55 or max(result['popfreqs'].STNE_1_0[100:])>55 or max(result['popfreqs'].Th_0_0[100:])>90 or max(result['popfreqs'].Th_1_0[100:])>90 or max(result['popfreqs'].LIP_0_0[100:])>100 or max(result['popfreqs'].LIP_1_0[100:])>100 or max(result['popfreqs'].FSI_0[100:])>70:
        print('The step', ns, 'causes firing rates greater than the expected')
        
    #Check Baselines in specific boundaries
    if sum(result['popfreqs'].D1STR_0_0[100:600])/500.0>5 or sum(result['popfreqs'].D1STR_1_0[100:600])/500.0>5 or sum(result['popfreqs'].D2STR_0_0[100:600])/500.0>5 or sum(result['popfreqs'].D2STR_1_0[100:600])/500.0>5 or sum(result['popfreqs'].GPeP_0_0[100:600])/500.0<40 or sum(result['popfreqs'].GPeP_1_0[100:600])/500.0<40 or sum(result['popfreqs'].GPeP_0_0[100:600])/500.0>90 or sum(result['popfreqs'].GPeP_1_0[100:600])/500.0>90 or sum(result['popfreqs'].GPi_0_0[100:600])/500.0<40 or sum(result['popfreqs'].GPi_1_0[100:600])/500.0<40 or sum(result['popfreqs'].GPi_0_0[100:600])/500.0>90 or sum(result['popfreqs'].GPi_1_0[100:600])/500.0>90 or sum(result['popfreqs'].STNE_0_0[100:600])/500.0<10 or sum(result['popfreqs'].STNE_1_0[100:600])/500.0<10 or sum(result['popfreqs'].STNE_0_0[100:600])/500.0>35 or sum(result['popfreqs'].STNE_1_0[100:600])/500.0>35 or sum(result['popfreqs'].Th_0_0[100:600])/500.0<4 or sum(result['popfreqs'].Th_1_0[100:600])/500.0<4 or sum(result['popfreqs'].Th_0_0[100:600])/500.0>20 or sum(result['popfreqs'].Th_1_0[100:600])/500.0>20 or sum(result['popfreqs'].FSI_0[100:600])/500.0<5 or sum(result['popfreqs'].FSI_0[100:600])/500.0>40:
        print('The step', ns, 'causes baselines out of range')

    print('-------------------------------------------------------------')


Step 0 Values considered [[0.01829281 0.01525626 0.01960396 0.05153759 0.09796664 0.22960938
  0.20020997 0.06215675 0.28       0.63948589 0.28       1.6439976
  0.33497288 0.01990827 0.01084666 0.09927002 0.2343329  0.26481109
  0.0320269  0.14559138 0.06774161 1.51       0.02834884 1.75      ]]
Waiting for subprocess to end
Done
The step 0 causes a reaction time out of bounds
-------------------------------------------------------------
Step 1 Values considered [[0.0217038  0.01773098 0.03455728 0.03843897 0.36115711 0.41299378
  0.15446778 0.07006857 0.28       0.62832576 0.28       1.35848586
  0.35910421 0.01800997 0.0113155  0.11731854 0.21426099 0.24846707
  0.02195388 0.13543113 0.06189485 1.51       0.02300516 1.75      ]]
Waiting for subprocess to end
Done
The step 1 causes firing rates greater than the expected
-------------------------------------------------------------
Step 2 Values considered [[0.01026771 0.01469028 0.03587288 0.05619276 0.09451541 0.52989191
  0.0362769

Waiting for subprocess to end
Done
The step 18 causes a reaction time out of bounds
-------------------------------------------------------------
Step 19 Values considered [[0.01020537 0.02111256 0.03187626 0.04623578 0.36701787 0.58737815
  0.04122276 0.3390327  0.28       0.59467643 0.28       1.68660621
  0.38177863 0.0179431  0.01087843 0.08526381 0.03094313 0.26333618
  0.00723396 0.19319714 0.06404967 1.51       0.02193484 1.75      ]]
Waiting for subprocess to end
Done
The step 19 causes a reaction time out of bounds
-------------------------------------------------------------
Step 20 Values considered [[0.01037521 0.02069999 0.0586756  0.04411118 0.29885634 0.4011272
  0.48979555 0.3889242  0.28       0.34268331 0.28       1.13201164
  0.3599749  0.01936014 0.01080447 0.11554999 0.15868182 0.2386309
  0.03081625 0.25926248 0.06435318 1.51       0.02959395 1.75      ]]
Waiting for subprocess to end
Done
The step 20 causes a reaction time out of bounds
--------------------------

Waiting for subprocess to end
Done
The step 37 causes a reaction time out of bounds
The step 37 causes firing rates greater than the expected
-------------------------------------------------------------
Step 38 Values considered [[0.01995196 0.01781695 0.02379694 0.02812508 0.59085774 0.20346645
  0.38313625 0.50486988 0.28       0.99345989 0.28       1.16156147
  0.35731237 0.01860536 0.01080094 0.11397932 0.11088788 0.25027914
  0.01240844 0.2454418  0.09146983 1.51       0.02137796 1.75      ]]
Waiting for subprocess to end
Done
The step 38 causes a reaction time out of bounds
-------------------------------------------------------------
Step 39 Values considered [[0.01196515 0.02000647 0.03073011 0.05356102 0.38478259 0.59298142
  0.57883592 0.36523505 0.28       0.556246   0.28       1.416532
  0.34185554 0.0185707  0.01120924 0.10666336 0.1128436  0.25930141
  0.02569429 0.22576184 0.08356399 1.51       0.03495642 1.75      ]]
Waiting for subprocess to end
Done
-----------------

Waiting for subprocess to end
Done
The step 56 causes a reaction time out of bounds
-------------------------------------------------------------
Step 57 Values considered [[0.02041361 0.02133602 0.05721108 0.01495982 0.2162842  0.28862826
  0.16069183 0.42574779 0.28       1.110468   0.28       1.24595475
  0.35374049 0.01808735 0.01108195 0.08030076 0.09490351 0.24741773
  0.02965303 0.38972473 0.07075581 1.51       0.03191224 1.75      ]]
Waiting for subprocess to end
Done
The step 57 causes a reaction time out of bounds
-------------------------------------------------------------
Step 58 Values considered [[0.01452694 0.01393564 0.03378774 0.03583496 0.33042581 0.62489439
  0.07006627 0.53549393 0.28       1.1698335  0.28       1.5122248
  0.3571812  0.01966595 0.01099251 0.09973884 0.2472485  0.25228548
  0.01687828 0.35630896 0.07345351 1.51       0.02125356 1.75      ]]
Waiting for subprocess to end
Done
The step 58 causes a reaction time out of bounds
The step 58 causes firing

Waiting for subprocess to end
Done
-------------------------------------------------------------
Step 75 Values considered [[0.01556133 0.01591227 0.04671371 0.02855444 0.55843743 0.14120391
  0.23527618 0.16808793 0.28       0.90536521 0.28       1.6705255
  0.38826912 0.01789176 0.01121752 0.11154388 0.0529939  0.26451124
  0.0347948  0.14796757 0.06674487 1.51       0.02133653 1.75      ]]
Waiting for subprocess to end
Done
-------------------------------------------------------------
Step 76 Values considered [[0.02129697 0.01719143 0.04160468 0.02610142 0.07496384 0.62754763
  0.10249499 0.17216753 0.28       1.22968805 0.28       1.08755712
  0.3894103  0.01887367 0.01123314 0.08136272 0.1969063  0.25374225
  0.02309613 0.29042123 0.07040997 1.51       0.02554912 1.75      ]]
Waiting for subprocess to end
Done
The step 76 causes a reaction time out of bounds
The step 76 causes baselines out of range
-------------------------------------------------------------
Step 77 Values cons

...

Waiting for subprocess to end
Done
The step 207 causes firing rates greater than the expected
-------------------------------------------------------------
Step 208 Values considered [[0.01379116 0.01600501 0.03360626 0.0548545  0.42904703 0.15843505
  0.50459325 0.11663989 0.28       0.31085758 0.28       1.03915491
  0.3620568  0.01812881 0.01131694 0.10026393 0.11500545 0.25143262
  0.0277015  0.27484792 0.05211852 1.51       0.0217293  1.75      ]]
Waiting for subprocess to end
Done
-------------------------------------------------------------
Step 209 Values considered [[0.01753982 0.02046808 0.01476128 0.04347757 0.09261302 0.51808265
  0.50322831 0.55824444 0.28       0.41384993 0.28       1.76995123
  0.37136992 0.01745989 0.01119414 0.1085243  0.05841015 0.23927932
  0.02623552 0.28091368 0.06623179 1.51       0.02790575 1.75      ]]
Waiting for subprocess to end
Done
-------------------------------------------------------------
Step 210 Values considered [[0.01555217 0.019358

Waiting for subprocess to end
Done
The step 226 causes a reaction time out of bounds
-------------------------------------------------------------
Step 227 Values considered [[0.01919832 0.01521779 0.04479917 0.03293181 0.43166242 0.07325145
  0.56432079 0.17852902 0.28       1.1365342  0.28       1.10305237
  0.37338641 0.01791406 0.0111499  0.10242095 0.16954441 0.24072747
  0.03487128 0.39835359 0.09683694 1.51       0.03118718 1.75      ]]
Waiting for subprocess to end
Done
The step 227 causes a reaction time out of bounds
-------------------------------------------------------------
Step 228 Values considered [[0.01658813 0.01622156 0.03559375 0.02679968 0.53911667 0.20616714
  0.47317863 0.57142338 0.28       0.65250244 0.28       1.29040681
  0.34894436 0.01923453 0.01098708 0.09431868 0.18095337 0.25491084
  0.0105983  0.16579787 0.08834339 1.51       0.03457012 1.75      ]]
Waiting for subprocess to end
Done
The step 228 causes a reaction time out of bounds
-------------------

Waiting for subprocess to end
Done
The step 245 causes a reaction time out of bounds
The step 245 causes firing rates greater than the expected
-------------------------------------------------------------
Step 246 Values considered [[0.01347105 0.01156955 0.02355242 0.05822152 0.51039483 0.23966251
  0.47732434 0.20798332 0.28       1.21958551 0.28       1.28681674
  0.3616361  0.01976556 0.01140601 0.08461583 0.08617097 0.25719214
  0.03356903 0.17407403 0.05609808 1.51       0.02566078 1.75      ]]
Waiting for subprocess to end
Done
The step 246 causes a reaction time out of bounds
-------------------------------------------------------------
Step 247 Values considered [[0.0126649  0.01370302 0.02054158 0.04585912 0.47901599 0.07819924
  0.04605601 0.31948483 0.28       0.84887102 0.28       1.22881548
  0.35580643 0.01852257 0.01147688 0.09635807 0.0602421  0.23781304
  0.03884073 0.22868698 0.07860202 1.51       0.03360295 1.75      ]]
Waiting for subprocess to end
Done
The step 2

Waiting for subprocess to end
Done
The step 264 causes a reaction time out of bounds
-------------------------------------------------------------
Step 265 Values considered [[0.01127195 0.01529108 0.02148142 0.03518646 0.56929397 0.58583617
  0.36797005 0.50842805 0.28       0.90867239 0.28       1.42880576
  0.36357222 0.01866474 0.01095884 0.09099388 0.01396664 0.24821885
  0.03522103 0.23245668 0.06368461 1.51       0.02848419 1.75      ]]
Waiting for subprocess to end
Done
The step 265 causes a reaction time out of bounds
-------------------------------------------------------------
Step 266 Values considered [[0.01953576 0.01778475 0.05534489 0.04908245 0.17443441 0.24204442
  0.28751304 0.39201941 0.28       0.48813532 0.28       1.07300111
  0.3819262  0.01772089 0.0111865  0.09979811 0.17413872 0.24999085
  0.00908746 0.36196061 0.08552629 1.51       0.02421778 1.75      ]]
Waiting for subprocess to end
Done
The step 266 causes firing rates greater than the expected
----------

Done
The step 282 causes a reaction time out of bounds
-------------------------------------------------------------
Step 283 Values considered [[0.01938988 0.01227487 0.03974773 0.05235576 0.29490541 0.17817618
  0.22405411 0.5006425  0.28       0.24863235 0.28       1.60672957
  0.33775802 0.01938154 0.01118478 0.11671496 0.14078702 0.24613958
  0.01609042 0.183077   0.05222978 1.51       0.03440709 1.75      ]]
Waiting for subprocess to end
Done
-------------------------------------------------------------
Step 284 Values considered [[0.02032164 0.01487338 0.02123262 0.02120465 0.40531185 0.09631691
  0.32042633 0.44233003 0.28       0.93725421 0.28       1.01584766
  0.33038083 0.01902966 0.01120149 0.08383053 0.19511379 0.257256
  0.01383146 0.26264695 0.06910472 1.51       0.03226698 1.75      ]]
Waiting for subprocess to end
Done
The step 284 causes a reaction time out of bounds
-------------------------------------------------------------
Step 285 Values considered [[0.01441174

##### Creat the CBGT output table of mean FR

In [9]:
NumSteps=300
FRdatatable=[]
for ns in range(0,NumSteps):
#    if ns not in [11,43,75]:
    foldername="Uniform_Test"+str(ns)
    (paramtable,datatables) = loadData("",foldername)
    
    datatables2=datatables[0].loc[datatables[0]['decision']!=None]
    dtaux=datatables2.mean()
    di=dtaux.to_dict()

    FRdatatable.append(di)

FRdatatable=pd.DataFrame(FRdatatable)
targetfolder = os.path.join("","CBGTSolution")
file_name = os.path.join(targetfolder,"CBGTSol.pkl")
if not os.path.exists(targetfolder):
        os.makedirs(targetfolder)
FRdatatable.to_pickle(file_name)